# Chapter 15: Processing Sequences Using RNNs and CNNs

## Exercise 10

Download the [Bach chorales](https://homl.info/bach) dataset and unzip it. It is composed of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note's index on a piano (except for the value 0, which means that no note is played). Train a model—recurrent, convolutional, or both—that can predict the next time step (four notes), given a sequence of time steps from a chorale. Then use this model to generate Bach-like music, one note at a time: you can do this by giving the model the start of a chorale and asking it to predict the next time step, then appending these time steps to the input sequence and asking the model for the next note, and so on. Also make sure to check out [Google's Coconet model](https://homl.info/coconet), which was used for a nice [Google doodle about Bach](https://www.google.com/doodles/celebrating-johann-sebastian-bach).

In [173]:
from pathlib import Path
from tensorflow import keras

import numpy as np
import pandas as pd
import tensorflow as tf

In [174]:
root = 'https://github.com/ageron/handson-ml2/raw/master/datasets/jsb_chorales/'
filename = 'jsb_chorales.tgz'
filepath = keras.utils.get_file(filename, root+filename, cache_subdir='datasets/jsb_chorales', extract=True)

In [175]:
path = Path(filepath).parent
train_files = sorted(path.glob('train/chorale_*.csv'))
valid_files = sorted(path.glob('valid/chorale_*.csv'))
test_files  = sorted(path.glob('test/chorale_*.csv'))

In [176]:
def load_chorales(files):
    return [pd.read_csv(file).values.astype(np.uint8) for file in files]

In [177]:
def transform_chorale(chorale, window_size=128, min_note=36):
    chorale = chorale.reshape(-1)
    chorale = np.where(chorale==0, chorale, chorale-min_note+1)
    assert len(chorale) >= window_size
    batches = np.empty((len(chorale) - window_size, window_size), np.uint8)
    for i in range(len(chorale) - window_size):
        batches[i,:] = chorale[i:i+window_size]
    return batches

In [178]:
def load_data(files, window_size=128, min_note=36):
    data = load_chorales(files)
    data = [transform_chorale(chorale, window_size, min_note) for chorale in data]
    X = np.vstack([batches[:-1] for batches in data])
    Y = np.vstack([batches[1:] for batches in data])
    return X, Y

In [179]:
X_train, Y_train = load_data(train_files)
X_valid, Y_valid = load_data(valid_files)

## Simple LSTM model

In [180]:
embedding_dim = 5
lstm_dim = 256
input_dim = len(np.unique(X_train))

In [169]:
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_dim, input_shape=[None]),
    keras.layers.LSTM(lstm_dim, return_sequences=True),
    keras.layers.LSTM(lstm_dim, return_sequences=True),
    keras.layers.Dense(input_dim, activation='softmax')
])

In [170]:
optimizer = keras.optimizers.Nadam(lr=1e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [171]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 5)           235       
_________________________________________________________________
lstm_9 (LSTM)                (None, None, 256)         268288    
_________________________________________________________________
lstm_10 (LSTM)               (None, None, 256)         525312    
_________________________________________________________________
dense_4 (Dense)              (None, None, 47)          12079     
Total params: 805,914
Trainable params: 805,914
Non-trainable params: 0
_________________________________________________________________


In [172]:
history = model.fit(X_train, Y_train, batch_size=32, epochs=20, validation_data=(X_valid, Y_valid))

Epoch 1/20
5981/5981 [==============================] - 145s 24ms/step - loss: 0.5329 - accuracy: 0.8483 - val_loss: 0.9532 - val_accuracy: 0.7918
Epoch 2/20
5981/5981 [==============================] - 140s 23ms/step - loss: 0.1351 - accuracy: 0.9600 - val_loss: 1.0632 - val_accuracy: 0.7901
Epoch 3/20
5981/5981 [==============================] - 157s 26ms/step - loss: 0.1162 - accuracy: 0.9646 - val_loss: 1.1206 - val_accuracy: 0.7918
Epoch 4/20
5981/5981 [==============================] - 131s 22ms/step - loss: 0.1080 - accuracy: 0.9667 - val_loss: 1.1524 - val_accuracy: 0.7919
Epoch 5/20
5981/5981 [==============================] - 133s 22ms/step - loss: 0.1029 - accuracy: 0.9680 - val_loss: 1.1915 - val_accuracy: 0.7920
Epoch 6/20
5981/5981 [==============================] - 131s 22ms/step - loss: 0.0996 - accuracy: 0.9688 - val_loss: 1.2151 - val_accuracy: 0.7914
Epoch 7/20
5981/5981 [==============================] - 132s 22ms/step - loss: 0.0971 - accuracy: 0.9695 - val_loss: 1

In [197]:
model.save('models/ch15_ex10_simple.h5')

In [185]:
## functions to play chorales

In [ ]:
from IPython.display import Audio

def notes_to_frequencies(notes):
    # Frequency doubles when you go up one octave; there are 12 semi-tones
    # per octave; Note A on octave 4 is 440 Hz, and it is note number 69.
    return 2 ** ((np.array(notes) - 69) / 12) * 440

def frequencies_to_samples(frequencies, tempo, sample_rate):
    note_duration = 60 / tempo # the tempo is measured in beats per minutes
    # To reduce click sound at every beat, we round the frequencies to try to
    # get the samples close to zero at the end of each note.
    frequencies = np.round(note_duration * frequencies) / note_duration
    n_samples = int(note_duration * sample_rate)
    time = np.linspace(0, note_duration, n_samples)
    sine_waves = np.sin(2 * np.pi * frequencies.reshape(-1, 1) * time)
    # Removing all notes with frequencies ≤ 9 Hz (includes note 0 = silence)
    sine_waves *= (frequencies > 9.).reshape(-1, 1)
    return sine_waves.reshape(-1)

def chords_to_samples(chords, tempo, sample_rate):
    freqs = notes_to_frequencies(chords)
    freqs = np.r_[freqs, freqs[-1:]] # make last note a bit longer
    merged = np.mean([frequencies_to_samples(melody, tempo, sample_rate)
                     for melody in freqs.T], axis=0)
    n_fade_out_samples = sample_rate * 60 // tempo # fade out last note
    fade_out = np.linspace(1., 0., n_fade_out_samples)**2
    merged[-n_fade_out_samples:] *= fade_out
    return merged

def play_chords(chords, tempo=160, amplitude=0.1, sample_rate=44100, filepath=None):
    samples = amplitude * chords_to_samples(chords, tempo, sample_rate)
    if filepath:
        from scipy.io import wavfile
        samples = (2**15 * samples).astype(np.int16)
        wavfile.write(filepath, sample_rate, samples)
        return display(Audio(filepath))
    else:
        return display(Audio(samples, rate=sample_rate))

In [181]:
def generate_chorale(model, seed_chords, length, temperature=1, min_note=36):
    arpegio = seed_chords
    for chord in range(length):
        next_note_probas = model.predict(arpegio)[-1, -1]
        rescaled_logits = np.log(next_note_probas) / temperature
        next_note_probas = np.exp(rescaled_logits)
        next_note_probas /= np.sum(next_note_probas)
        next_note = np.random.choice(input_dim, size=1, p=next_note_probas)
        arpegio = np.concatenate((arpegio, next_note))
    arpegio = np.where(arpegio==0, arpegio, arpegio+min_note-1)
    chorale = arpegio.reshape(-1, 4)
    return chorale

In [186]:
## Generate new chorales

In [187]:
idx_seed = np.random.randint(X_test.shape[0])
seed_chords = X_test[idx_seed, :8]
new_chorale = gen_chorale(model, seed_chords, 128-len(seed_chords))

In [189]:
play_chords(new_chorale)

In [190]:
## Second try, let's use a small WaveNet

In [192]:
model2 = keras.models.Sequential([
    keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_dim,
                           input_shape=[None]),
    keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(128, kernel_size=2, padding="causal", activation="relu", dilation_rate=16),
    keras.layers.BatchNormalization(),
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(input_dim, activation="softmax")
])

In [193]:
optimizer = keras.optimizers.Nadam(lr=1e-3)
model2.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [194]:
history2 = model2.fit(X_train, Y_train, batch_size=32, epochs=20, validation_data=(X_valid, Y_valid))

Epoch 1/20
5981/5981 [==============================] - 145s 24ms/step - loss: 0.3954 - accuracy: 0.8870 - val_loss: 0.9512 - val_accuracy: 0.7955
Epoch 2/20
5981/5981 [==============================] - 142s 24ms/step - loss: 0.1712 - accuracy: 0.9485 - val_loss: 1.0813 - val_accuracy: 0.7970
Epoch 3/20
5981/5981 [==============================] - 149s 25ms/step - loss: 0.1509 - accuracy: 0.9537 - val_loss: 1.1484 - val_accuracy: 0.7985
Epoch 4/20
5981/5981 [==============================] - 149s 25ms/step - loss: 0.1413 - accuracy: 0.9563 - val_loss: 1.1947 - val_accuracy: 0.8021
Epoch 5/20
5981/5981 [==============================] - 148s 25ms/step - loss: 0.1350 - accuracy: 0.9580 - val_loss: 1.2180 - val_accuracy: 0.7999
Epoch 6/20
5981/5981 [==============================] - 151s 25ms/step - loss: 0.1303 - accuracy: 0.9593 - val_loss: 1.2380 - val_accuracy: 0.7985
Epoch 7/20
5981/5981 [==============================] - 147s 25ms/step - loss: 0.1267 - accuracy: 0.9603 - val_loss: 1

In [196]:
model2.save('models/ch15_ex10_wavenet.h5')